<a href="https://colab.research.google.com/github/fawazshah/News-Media-Reliability/blob/master/train_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd
import requests

In [40]:
TEXT_TYPE = 'headline' # 'headline' or 'body'

### Loading data

In [45]:
corpus_url = 'https://raw.githubusercontent.com/fawazshah/News-Media-Reliability/master/data/emnlp18/corpus-modified.tsv'

corpus = pd.read_csv(corpus_url, sep='\t')
urls = corpus['source_url_normalized'].values

# Ground truths
biases = corpus['bias'].values

765


In [3]:
article_data_json_url = 'https://raw.githubusercontent.com/fawazshah/News-Media-Reliability/master/data/scraped_articles.json'

r = requests.get(article_data_json_url)
article_data = r.json()

In [60]:
all_data_df = pd.DataFrame(columns=['article headline', 'article body', 'bias'])

article_headlines = []
article_bodies = []

news_sources_scraped = 0

for row in corpus.itertuples():
    url = row.source_url_normalized
    bias = row.bias
    if article_data["newspapers"][url] is not None:
        articles = article_data["newspapers"][url].get("articles", [])
        if len(articles) > 0:
            news_sources_scraped += 1
            for article in articles:
                article_headlines.append(article['title'])
                article_bodies.append(article['text'])
                all_data_df = all_data_df.append({'article headline': article['title'],
                                                  'article body': article['text'],
                                                  'bias': bias}, ignore_index=True)

In [85]:
all_data_df

,article headline,article body,bias
0,On the Ground at the Inauguration: The Only Th...,"Will Sennott\n\nWEDNESDAY, JANUARY 20, 2021, W...",left
1,"Under President Biden, Will the Yankees Return...",Thurman Munson and Reggie Jackson in 1977 From...,left
2,Gun Rights Absolutists Celebrate Martin Luther...,"Will Sennott\n\nMONDAY, JANUARY 18, 2021, RICH...",left
3,Thugs in Blue,"THE BEAT GOES ON … AND ON\n\nOnce Again, Polic...",left
4,HELL YEAH! Sheriff Clark Publicly DISEMBOWELS ...,Al Sharpton always has had a couple screws loo...,right
...,...,...,...
1649,UK Educators Rank-and-File Safety Committee di...,The UK Educators Rank-and-File Safety Committe...,left
1650,Make It Sing,Before I lay into the Democrats for missed opp...,left
1651,Bill Maher: The SPIN Interview,If you care at all about democracy and the way...,left
1652,Stephan Jenkins on What Culture Truly Means,"“When bad men combine, the good must associate...",left


### Text preprocessing

In [41]:
# Text preprocessing performed on both article headline and article body